In [2]:
//@ts-check

class MyArray {
  _idx_slice;
  constructor(flat, shape) {
    this.flat = flat;
    this.shape = shape;
    this.ravel = undefined;
    this.slice = undefined;
    this.reshape = undefined;
    this.from_js_array = undefined;
    this.asarray = undefined;
    this._binary_operation = undefined;
    this._reduce = undefined;
    this.sum = undefined;
    this.arange = undefined;
    this.zeros = undefined;
    this.ones = undefined;
    this._new = undefined;
    this.array = undefined;
    this.to_js_array = undefined;
    this.max = undefined;
    this.min = undefined;
    this.argmin = undefined;
    this.argmax = undefined;
    this.mean = undefined;
    this.var = undefined;
    this.product = undefined;
    this.std = undefined;
    this.add = undefined;
    this.subtract = undefined;
    this.multiply = undefined;
    this.divide = undefined;
    this.pow = undefined;
    this.mod = undefined;
    this._binary_operations = undefined;
    this._parse_sliceSpec = undefined;
    this._parse_sliceRange = undefined;
    this.__parse_sliceSpec = undefined;
    this.__parse_sliceRange = undefined;
    this.__shape_shifts = undefined;
    this._broadcast_shape = undefined;
    this._broadcast_shapes = undefined;
    this.__as_bool = undefined;
    this.__parse_list = undefined;
    this.__parse_shape = undefined;
  }
}

In [111]:
//@ts-check
var ohm = require('ohm-js');
var { spawnSync } = require('child_process')

// console.log(np`np.arange(120).reshape([-1,3])`)
// console.log(np`np.arange(120).reshape([-1, 3])[2:-3:6]`)


function npTest(htmlTemplateString, ...variables) {
  let idx = 0;
  const str = htmlTemplateString.join('###').replace(/###/g, () => {
    let value = variables[idx++];
    let out = JSON.stringify(value)
    if (Array.isArray(value)) out = `np.array(${out})`;
    return out;
  });
  const program = `
import numpy as np
import json
out = ${str}
if isinstance(out, np.ndarray):
    out = out.tolist()
print(json.dumps(out), flush=True)
`
  const process = spawnSync('python3', ['-c', program]);
  const stdout = process.stdout.toString();
  if (!stdout.length) throw new Error(process.stderr.toString());
  const expected = JSON.parse(stdout);
  const obtained = np(htmlTemplateString, ...variables);
  if (JSON.stringify(obtained) !== JSON.stringify(expected)) {
    console.error('ERROR. Mismatch for', str);
    console.error(JSON.stringify(expected));
    console.error(JSON.stringify(obtained));
  }
  return obtained;
}

/**
 * @param {TemplateStringsArray} htmlTemplateString
 * @param {any[]} variables
 * */
function np(htmlTemplateString, ...variables) {
  // Make a tree of operations
  // Join with #0# #1# #2#, ...
  // First by parenthesis () and square brackets []
  let idx = 0;
  const str = htmlTemplateString.join('###').replace(/###/g, () => `#${idx++}#`);
  // console.log(str);
  semanticVariables = variables;
  const match = myGrammar.match(str);
  if (!match.succeeded()) throw new Error(match.message);
  return semantics(match).eval();
}
var semanticVariables = [];


MyArray.prototype._new = function (shape, f) {
  shape = MyArray.prototype.__parse_shape(shape)
  const size = shape.reduce((a, b) => a * b, 1);
  const flat = Array.from({ length: size }, f)
  return new MyArray(flat, shape);
};

MyArray.prototype.__parse_shape = function (list) {
  if (Array.isArray(list)) return list;
  if (list instanceof MyArray) {
    if (list.shape.length > 1) {
      throw new Error(`Expected flat list. Got array with shape ${list.shape}`);
    }
    return list.flat;
  }
  if (typeof list == "number") return [list];
  throw new Error(`Expected list. Got ${list}`);
}
MyArray.prototype.zeros = function (shape) {
  return MyArray.prototype._new(shape, (_) => 0)
};
MyArray.prototype.ones = function (shape) {
  return MyArray.prototype._new(shape, (_) => 1)
};
MyArray.prototype.arange = function (arg0, arg1) {
  let start, end;
  if (arg1 === undefined) start = 0, end = arg0;
  else start = arg0, end = arg1;
  return MyArray.prototype._new(end - start, (_, i) => start + i)
};

// ==============================
//       Reducing functions
// ==============================

MyArray.prototype.__as_bool = function (obj) {
  if (typeof obj == 'string') throw new Error(`'string' object can not be interpreted as boolean: ${obj}`);
  return !!(0 + obj);
}
MyArray.prototype._reduce = function (arr, axis, keepdims, reducer) {
  keepdims = MyArray.prototype.__as_bool(keepdims);
  arr = MyArray.prototype.asarray(arr);
  if (axis == null) return reducer(arr.flat);
  if (axis < 0) axis = arr.shape.length - 1;
  let m = arr.shape[axis];
  let shift = MyArray.prototype.__shape_shifts(arr.shape)[axis];
  const groups = Array.from({ length: m }, (_) =>/**@type {number[]}*/([]));
  arr.flat.forEach((value, i) => groups[(Math.floor(i / shift)) % m].push(value));
  // Transpose it:
  let nCols = arr.flat.length / m;
  const groupsT = [];
  for (let j = 0; j < nCols; j++) {
    const newRow = [];
    for (let i = 0; i < m; i++) newRow.push(groups[i][j]);
    groupsT.push(newRow);
  }
  const flat = groupsT.map(reducer);
  let shape = [...arr.shape];
  if (keepdims) shape[axis] = 1;
  else shape = shape.filter((_, i) => i != axis);
  return new MyArray(flat, shape);
};

MyArray.prototype.sum = function (arr) {
  const { axis, keepdims } = Object.assign({ axis: null, keepdims: false }, this);
  return MyArray.prototype._reduce(arr, axis, keepdims, (arr) => arr.reduce((a, b) => a + b, 0));
};
MyArray.prototype.product = function (arr) {
  const { axis, keepdims } = Object.assign({ axis: null, keepdims: false }, this);
  return MyArray.prototype._reduce(arr, axis, keepdims, (arr) => arr.reduce((a, b) => a * b, 1));
};
MyArray.prototype.max = function (arr) {
  const { axis, keepdims } = Object.assign({ axis: null, keepdims: false }, this);
  return MyArray.prototype._reduce(arr, axis, keepdims, (arr) => Math.max(...arr));
};
MyArray.prototype.min = function (arr) {
  const { axis, keepdims } = Object.assign({ axis: null, keepdims: false }, this);
  return MyArray.prototype._reduce(arr, axis, keepdims, (arr) => Math.min(...arr));
};
MyArray.prototype.argmax = function (arr) {
  const { axis, keepdims } = Object.assign({ axis: null, keepdims: false }, this);
  return MyArray.prototype._reduce(arr, axis, keepdims, (arr) => arr.indexOf(Math.max(...arr)));
};
MyArray.prototype.argmin = function (arr) {
  const { axis, keepdims } = Object.assign({ axis: null, keepdims: false }, this);
  return MyArray.prototype._reduce(arr, axis, keepdims, (arr) => arr.indexOf(Math.min(...arr)));
};
MyArray.prototype.mean = function (arr) {
  const { axis, keepdims } = Object.assign({ axis: null, keepdims: false }, this);
  return MyArray.prototype._reduce(arr, axis, keepdims, (arr) => arr.reduce((a, b) => a + b, 0) / arr.length);
};
MyArray.prototype.var = function (arr) {
  const { axis, keepdims } = Object.assign({ axis: null, keepdims: false }, this);
  const mean = MyArray.prototype.mean.bind({ axis, keepdims: true })(arr);
  arr = MyArray.prototype.subtract(arr, mean);
  arr = MyArray.prototype.multiply(arr, arr);
  return MyArray.prototype.mean.bind({ axis, keepdims })(arr);
};
MyArray.prototype.std = function (arr) {
  const variance = MyArray.prototype.var.bind(this)(arr);
  return MyArray.prototype.pow(variance, 0.5);
};


// ==============================
//       Binary operations
// ==============================


MyArray.prototype._binary_operation = function (A, B, func) {
  if (typeof func === "string") {
    const symbol = func;
    const operators = MyArray.prototype._binary_operations;
    func = operators[symbol] || operators[symbol.slice(-1)];
    if (func === undefined) throw new Error(`Unknown operator ${symbol}`)
  }
  // Find output shape and input broadcast shapes
  A = MyArray.prototype.asarray(A);
  B = MyArray.prototype.asarray(B);
  const [shape, shapeA, shapeB] = MyArray.prototype._broadcast_shapes(A.shape, B.shape);
  const shiftsA = MyArray.prototype.__shape_shifts(shapeA);
  const shiftsB = MyArray.prototype.__shape_shifts(shapeB);
  // Iterate with broadcasted indices
  const result = new Array(shape.reduce((a, b) => a * b, 1)).fill(undefined);
  for (let i = 0; i < result.length; i++) {
    let idxA = 0, idxB = 0, idx = i;
    for (let axis = shape.length - 1; axis >= 0; axis--) {
      idxA += shiftsA[axis] * (idx % shapeA[axis]);
      idxB += shiftsB[axis] * (idx % shapeB[axis]);
      idx = Math.floor(idx / shape[axis]);
    }
    result[i] = func(A.flat[idxA], B.flat[idxB]);
  };
  return new MyArray(result, shape);
}

MyArray.prototype._binary_operations = {
  "+": (a, b) => a + b,
  "-": (a, b) => a - b,
  "*": (a, b) => a * b,
  "/": (a, b) => a / b,
  "%": (a, b) => (a % b),
  "**": (a, b) => Math.pow(a, b),
  "=": (a, b) => b,
  "maximum": (a, b) => Math.max(a, b),
  "minimum": (a, b) => Math.min(a, b),
};


MyArray.prototype._broadcast_shapes = function (shapeA, shapeB) {
  const shape = [];
  const maxDim = Math.max(shapeA.length, shapeB.length);
  shapeA = [...Array.from({ length: maxDim - shapeA.length }, () => 1), ...shapeA];
  shapeB = [...Array.from({ length: maxDim - shapeB.length }, () => 1), ...shapeB];
  for (let axis = maxDim - 1; axis >= 0; axis--) {
    const dim1 = shapeA[axis];
    const dim2 = shapeB[axis];
    if (dim1 !== 1 && dim2 !== 1 && dim1 !== dim2)
      throw new Error(`Can not broadcast axis ${axis} with sizes ${dim1} and ${dim2}`);
    shape.unshift(Math.max(dim1, dim2));
  }
  return [shape, shapeA, shapeB];
}

MyArray.prototype.add = function (A, B) {
  return MyArray.prototype._binary_operation(A, B, (a, b) => a + b);
};
MyArray.prototype.subtract = function (A, B) {
  return MyArray.prototype._binary_operation(A, B, (a, b) => a - b);
};
MyArray.prototype.multiply = function (A, B) {
  return MyArray.prototype._binary_operation(A, B, (a, b) => a * b);
};
MyArray.prototype.divide = function (A, B) {
  return MyArray.prototype._binary_operation(A, B, (a, b) => a / b);
};
MyArray.prototype.pow = function (A, B) {
  return MyArray.prototype._binary_operation(A, B, (a, b) => Math.pow(a, b));
};
MyArray.prototype.mod = function (A, B) {
  return MyArray.prototype._binary_operation(A, B, (a, b) => (a % b));
};



MyArray.prototype.asarray = function (A) {
  if (A instanceof MyArray) return A;
  else return MyArray.prototype.from_js_array(A);
}
MyArray.prototype.array = function (A) {
  if (A instanceof MyArray) return new MyArray([...A.flat], [...A.shape]);
  else return MyArray.prototype.from_js_array(A);
}
MyArray.prototype.from_js_array = function (arr) {
  if (typeof arr === "number") return new MyArray([arr], []);
  if (!Array.isArray(arr)) throw new Error(`Can't parse as array: ${arr}`);
  const shape = [];
  let root = arr;
  while (Array.isArray(root)) {
    shape.push(root.length);
    root = root[0];
    if (shape.length > 256) throw new Error(`Circular reference or excessive array depth`);
  }
  let dtype = root === true || root === false ? Boolean : Number;
  const flat = [];
  const pushToFlat = (arr, axis) => {
    // Check consistency
    if (axis == shape.length - 1) {
      for (let elem of arr) {
        if (Array.isArray(elem)) throw new Error(`Inconsistent shape`);
        flat.push(elem);
        // Update dtype
      }
    } else {
      if (!Array.isArray(arr)) throw new Error(`Inconsistent shape`);
      for (let sub of arr) {
        if (sub.length != shape[axis + 1])
          throw new Error(`Inconsistent shape: found sibling arrays of lengths ${sub.length} and ${shape[axis + 1]}`);
        pushToFlat(sub, axis + 1);
      }
    }
  }
  pushToFlat(arr, 0);
  return new MyArray(flat, shape)
}
MyArray.prototype.to_js_array = function (arr) {
  if (!(arr instanceof MyArray)) throw new Error(`Expected MyArray: ${arr}`);
  if (arr.shape.length == 0) return arr.flat[0];
  function recursiveReshape(flatArr, shapeArr) {
    if (shapeArr.length === 0) {
      return flatArr.shift();
    }
    const innerShape = shapeArr.slice(1);
    const outerSize = shapeArr[0];
    const innerArray = [];
    for (let i = 0; i < outerSize; i++) {
      innerArray.push(recursiveReshape(flatArr, innerShape));
    }
    return innerArray;
  }
  return recursiveReshape(arr.flat, arr.shape);
}
MyArray.prototype.ravel = function (A) {
  A = MyArray.prototype.asarray(A);
  return new MyArray([...A.flat], [A.flat.length]);
};

// =========================================
//     Slicing
// =========================================

MyArray.prototype.__parse_sliceRange = function (axis_size, { start, stop, step }) {
  if (start == null) start = 0;
  else if (start < 0) start = axis_size + start;
  if (stop == null) stop = axis_size;
  else if (stop < 0) stop = axis_size + stop;
  if (step == null) step = 1;
  else if (step == 0) throw new Error(`Slice range with step size of zero`);
  if (!isFinite(start) || !isFinite(stop) || !isFinite(step)) throw new Error(`Invalid slice ${[start, stop, step]}. Axis size ${axis_size}`);
  let indices = [];
  if (step > 0) {
    start = Math.max(start, 0);
    stop = Math.min(stop, axis_size);
    for (let i = start; i < stop; i += step) indices.push(i);
  } else {
    stop = Math.max(stop, 0);
    start = Math.min(start, axis_size);
    for (let i = start; i > stop; i += step) indices.push(i);
  }
  return indices;
}
MyArray.prototype.__parse_sliceSpec = function (shape, sliceSpec) {
  const nones = sliceSpec.map((_, i) => i).filter(i => sliceSpec[i] == "None");
  const _sliceSpec = sliceSpec.filter(v => v != "None");
  if (_sliceSpec.length > shape.length) throw new Error(`Invalid slice spec: ${JSON.stringify(sliceSpec)} for shape ${shape}`);
  sliceSpec = _sliceSpec;

  const collapsed = [];
  const slices = shape.map((_, axis) => {
    let slice = sliceSpec[axis];
    if (slice === undefined) slice = ":";
    if (slice == ':') return Array.from({ length: shape[axis] }, (_, i) => i);
    if (slice === parseInt(slice)) {
      collapsed.push(axis);
      return [slice];
    }
    if (slice.isRange) {
      return MyArray.prototype.__parse_sliceRange(shape[axis], slice);
    }
    if (!Array.isArray(slice)) throw `Expected array: ${slice}`;
    // TODO: if dtype is bool, do a "where"
    return slice;
  });
  // Post-process the shape (broadcast nones) and collapse integers:
  let outShape = slices.map(l => l.length);
  for (let i of collapsed) outShape[i] = null;
  for (let i of nones) outShape = [...outShape.slice(0, i), 1, ...outShape.slice(i)];
  outShape = outShape.filter(size => size !== null);
  return [outShape, slices];
}
MyArray.prototype.__shape_shifts = function (shape) {
  // increasing one by one on a given axis is increasing by shifts[axis] in flat representation
  const shifts = Array.from({ length: shape.length }, (_) => 0);
  shifts[shape.length - 1] = 1;
  for (let i = shape.length - 2; i >= 0; i--) shifts[i] = shifts[i + 1] * shape[i + 1];
  return shifts;
}

MyArray.prototype._idx_slice = function (shape, sliceSpec) {
  // Iterative cartesian product of the slices.
  const [outShape, slices] = MyArray.prototype.__parse_sliceSpec(shape, sliceSpec);
  if (slices.map(l => l.length).reduce((a, b) => a * b, 1) == 0) {
    return [outShape, []];
  }
  const shifts = MyArray.prototype.__shape_shifts(shape);
  const indices = [];
  const maxIndex = slices.length - 1;
  const tuple = new Array(slices.length).fill(0);
  let current = shifts.map((v, i) => v * slices[i][0]).reduce((a, b) => a + b, 0);
  while (true) {
    // const currentTuple = tuple.map((index, arrayIndex) => slices[arrayIndex][index]);
    indices.push(current);
    // Increment the rightmost index
    let i = ++tuple[maxIndex];
    if (i < slices[maxIndex].length) {
      current += (slices[maxIndex][i] - slices[maxIndex][i - 1]) * shifts[maxIndex];
    }
    // Check for overflow in each dimension
    for (let i = maxIndex; i > 0; i--) {
      if (tuple[i] === slices[i].length) {
        tuple[i] = 0;
        let j = ++tuple[i - 1];
        current -= (slices[i][slices[i].length - 1] - slices[i][0]) * shifts[i];
        if (j < slices[i - 1].length) current += (slices[i - 1][j] - slices[i - 1][j - 1]) * shifts[i - 1];
      }
    }
    if (!isFinite(current)) throw `Programming error`
    // Check if we have finished iterating
    if (tuple[0] === slices[0].length) break;
  }
  return [outShape, indices];
}


MyArray.prototype.slice = function (arr, sliceSpec) {
  const [shape, indices] = MyArray.prototype._idx_slice(arr.shape, sliceSpec);
  return new MyArray(indices.map(i => arr.flat[i]), shape);
}

var myGrammar = ohm.grammar(String.raw`
ArrayGrammar {
  Instruction
  = Variable AssignSymbol ArithmeticExp               -- directAssignment
  | Variable "[" Slice "]" AssignSymbol ArithmeticExp -- sliceAssignment
  | ArithmeticExp                       -- expression
   
  Arr
    = "(" ArithmeticExp ")"  -- parent
    | Arr "." Name CallArgs     -- method
    | Arr "." Name              -- attribute
    | Arr "[" Slice "]"         -- slice
    | Name ("." Name)* CallArgs -- call
    | number
    | Variable
    | "-" Arr   -- neg
    | "+" Arr   -- pos
  
  Variable
   = "#" digit+ "#"
  
  AssignSymbol
  ="="|"+="|"-="|"/="|"%="|"&="|"|="|"^="|"@="
    
  ArithmeticExp = AddExp
  AddExp
    = AddExp "+" MulExp  -- add
    | AddExp "-" MulExp  -- subtract
    | MulExp
  MulExp
    = MulExp "*" PowExp  -- multiply
    | MulExp "/" PowExp  -- divide
    | PowExp
  PowExp
    = PowExp "**" PowExp  -- pow
    | Arr
   
  Name  (an identifier)
    = (letter|"_") (letter|"_"|digit)*

  number  (a number)
    = digit* "." digit+  -- fract
    | digit+             -- whole
  
  int (an integer) = ""  digit+ | "-" digit+ | "+" digit+

  CallArgs // Using empty strings instead of separate rules
   = "(" Args ","  KwArgs ","? ")"
   | "(" Args ","? ""      ""  ")"
   | "(" ""   ","? KwArgs ","? ")"
   | "(" ""    ""  ""      ""  ")"
   
  Args = NonemptyListOf<ArgValue, ",">
  KwArgs = NonemptyListOf<KwArg, ",">
  KwArg = Name "=" ArgValue

  ArgValue = Constant | JsArray | ArithmeticExp
  Constant = "True" | "False" | "None" | "np.nan" | "np.inf" | String
  JsArray = "[" ListOf<ArgValue, ","> ","? "]"

  String = "\'" any* "\'" | "\"" any* "\""
   
  Slice = NonemptyListOf<SliceTerm, ",">
  SliceTerm
    = SliceRange
    | (":" | "..." | "None") -- constant
    | JsArray
    | ArithmeticExp
  
  SliceRange
    = int ":" int ":" int
    | int ":" int ""  ""
    | int ":" ""  ":" int
    | int ":" ""  ""  ""
    | ""  ":" int ":" int
    | ""  ":" int ""  ""
    | ""  ":" ""  ":" int
    | ""  ":" ""  ""  ""
}
`);

var semantics = myGrammar.createSemantics();
semanticVariables = [];

semantics.addOperation('eval', {
  number(digits) {
    return parseInt(digits.sourceString)
  },
  Arr_slice($arr, _open, $sliceSpec, _close) {
    const arr = $arr.eval();
    const sliceSpec = $sliceSpec.eval();
    return MyArray.prototype.slice(arr, sliceSpec);
  },
  SliceTerm_constant($x) {
    return $x.sourceString;
  },
  Arr_call($name, $names, _, $callArgs) {
    let name = $name.sourceString + $names.sourceString;
    if (name.slice(0, 3) == "np.") name = name.slice(3);
    const func = MyArray.prototype[name];
    if (func === undefined) throw new Error(`Unrecognized function ${name}`)
    const { args, kwArgs } = $callArgs.eval();
    return func.bind(kwArgs)(...args);
  },
  Arr_method($arr, _dot, $name, $callArgs) {
    let arr = $arr.eval();
    let name = $name.sourceString;
    if (name.slice(0, 3) == "np.") name = name.slice(3);
    const func = MyArray.prototype[name];
    if (func === undefined) throw new Error(`Unrecognized method ${name}`)
    const { args, kwArgs } = $callArgs.eval();
    return func.bind(kwArgs)(arr, ...args);
  },
  Arr_neg(_, $arr) { return MyArray.prototype.multiply(-1, $arr.eval()); },
  Arr_pos(_, $arr) { return $arr.eval(); },
  Arr_parent(_, $arr, __) { return $arr.eval(); },
  Arr_attribute($arr, _, $name) { return $arr.eval()[$name.sourceString]; },
  Variable(_, $i, __) {
    const i = parseInt($i.sourceString);
    const arr = semanticVariables[i];
    return arr;
  },
  int($sign, $value) {
    const value = parseInt($value.sourceString);
    if ($sign.sourceString == '-') return -value;
    else return value
  },
  Instruction_sliceAssignment($tgt, _open, $sliceSpec, _close, $symbol, $src) {
    // WARNING: Creates a copy. This is terrible for arr[2, 4, 3] = 5
    const tgt = $tgt.eval();
    const symbol = $symbol.sourceString;
    const sliceSpec = $sliceSpec.eval();
    const src = MyArray.prototype.asarray($src.eval());
    let arr = MyArray.prototype.asarray(tgt);
    let [shape, indices] = MyArray.prototype._idx_slice(arr.shape, sliceSpec);
    let result = new MyArray(indices.map(i => arr.flat[i]), shape);
    result = MyArray.prototype._binary_operation(result, src, symbol);
    if (indices.length != result.flat.length) throw new Error(`Can't assign size ${result.flat.length} to slice of size ${indices.length}`);
    for (let i of indices) arr.flat[i] = result.flat[i];
    arr = MyArray.prototype.to_js_array(arr);
    while (tgt.length) tgt.pop();
    tgt.push(...arr);
    return null;
  },
  Instruction_directAssignment($tgt, $slice, $src) {
    throw 'not implemented';
  },
  Instruction_expression($arr) {
    const arr = $arr.eval();
    if (typeof arr === "number") return arr;
    if (Array.isArray(arr)) return arr;
    return MyArray.prototype.to_js_array(arr);
  },
  AddExp_add($x, $symbol, $y) { return MyArray.prototype.add($x.eval(), $y.eval()); },
  AddExp_subtract($x, $symbol, $y) { return MyArray.prototype.subtract($x.eval(), $y.eval()); },
  MulExp_multiply($x, $symbol, $y) { return MyArray.prototype.multiply($x.eval(), $y.eval()); },
  MulExp_divide($x, $symbol, $y) { return MyArray.prototype.divide($x.eval(), $y.eval()); },
  PowExp_pow($x, $symbol, $y) { return MyArray.prototype.pow($x.eval(), $y.eval()); },

  SliceRange($start, _, $stop, __, $step) {
    const start = $start.eval();
    const stop = $stop.eval();
    const step = $step.eval();
    return { start, stop, step, isRange: true };
  },

  CallArgs(_open, $args, _comma, $kwArgs, _trailing, _close) {
    const args = $args.eval() || [];
    let parse_constants = (s) => {
      switch (s) {
        case "True": return true;
        case "False": return false;
        case "None": return null;
      }
      if (s.length && s[0] == '"' && s[s.length - 1] == '"') return s;
      if (s.length && s[0] == "'" && s[s.length - 1] == "'") return s;
      return s;
    }
    let entries = $kwArgs.eval() || [];
    entries = entries.map(([k, v]) => [k, parse_constants(v)]);
    let kwArgs = Object.fromEntries(entries);
    return { args, kwArgs };
  },
  KwArg($key, _equals, $value) {
    const key = $key.sourceString;
    const value = $value.sourceString;
    return [key, value];
  },
  NonemptyListOf(first, _, more) {
    return [first, ...more.children].map(c => c.eval());
  },
  JsArray(_open, $list, _trailing, _close) {
    const list = $list.eval();
    // Downcast arrays (needed because, e.g., for [-1, 3, -2], -1 and -2 are interpreted as MyArray rather than int)
    for (let i in list) if (list[i] instanceof MyArray) list[i] = MyArray.prototype.to_js_array(list[i]);
    return list;
  },
  _terminal() { return null; },
});


MyArray.prototype.reshape = function (A, shape) {
  shape = [...shape];
  const n = A.flat.length;
  const inferredIndex = shape.indexOf(-1);
  if (inferredIndex !== -1) {
    const productOfKnownDims = shape.filter(dim => dim !== -1).reduce((acc, val) => acc * val, 1);
    if (n % productOfKnownDims !== 0) {
      throw new Error("Invalid shape. The total number of elements must match the product of the known dimensions.");
    }
    shape[inferredIndex] = n / productOfKnownDims;
  }
  return new MyArray([...A.flat], shape);
};




[ [ 10, 11 ] ]

In [121]:
// Unit tests:

npTest`np.arange(120).reshape([-1,3])`
npTest`np.zeros(3)`
npTest`np.zeros(${[3, 5, 4]})`
npTest`np.sum(np.ones(${[10, 2]}), axis=0, keepdims=True) + np.arange(2)`
npTest`np.ravel(np.reshape( np.arange(120), ${[2, 3, 4, 5]} ))`
npTest`np.ravel(np.reshape( np.arange(120), ${[2, 3, 4, 5]} )[ :, 0, ${[1, 2]}, : ])`
npTest`np.ravel([[1,2], [3, 4]])`
npTest`np.arange(120).reshape([2,3,4,5])[:,0,[1,2],None,:].shape`
npTest`np.arange(120).reshape([2,3,4,5])[:,0,[1,2],:].shape`
npTest`np.arange(120).std()`

var a = npTest`np.array([[1,2], [3,6], [9, 10]])`
npTest`${a}.sum(axis=0)`
npTest`${a}.sum(axis=1)`
var a = npTest`np.array([[1,2], [2,4], [3,6]])`
npTest`${a}.sum(axis=0)`
npTest`${a}.sum(axis=1)`
npTest`${a}.sum(axis=0, keepdims=True)`
npTest`${a}.sum(axis=1, keepdims=True)`
npTest`${a}.sum(axis=0, keepdims=False)`

var a = npTest`np.arange(12).reshape([2, 3, 2])`
npTest`${a}.sum(axis=0)`
npTest`${a}.sum(axis=1)`
npTest`${a}.sum(axis=-1)`

var a = npTest`np.arange(6).reshape([2, 3])`
npTest`${a} * ${a}`
npTest`2 * ${a}`
npTest`-(${a} * ${a})`
npTest`-(${a} * ${a}).sum(axis=0, keepdims=True)`
npTest`-${a} * ${a}.sum(axis=0, keepdims=True)`
npTest`-(${a} * ${a}).sum(axis=-1, keepdims=True)`

npTest`np.sum(np.ones(np.array([10,2])), axis=0, keepdims=True) + np.arange(2)`

[ [ 10, 11 ] ]